# 교차 검증 및 하이퍼파라미터 튜닝

**학습 목표:**
- 단일 `train_test_split`의 한계를 이해하고, 모델의 일반화 성능을 더 안정적으로 평가하기 위한 **K-Fold 교차 검증(K-Fold Cross-Validation)**을 학습합니다.
- 모델의 성능을 극대화하기 위해, 최적의 하이퍼파라미터 조합을 체계적으로 탐색하는 **그리드 서치(Grid Search)**를 사용합니다.
- 이 과정을 통해 모델 튜닝의 중요성과 표준적인 절차를 익힙니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### (1) 데이터 준비 및 전처리
이전과 동일하게 유방암 데이터셋을 사용하고, SVM 모델의 성능을 위해 데이터를 표준화합니다.

In [ ]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# GridSearchCV와 비교를 위해 전체 데이터를 사용합니다.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### (2) 교차 검증 (Cross-Validation)
데이터를 한 번만 나누어 평가하면, 우연히 테스트 세트가 쉽거나 어렵게 구성되어 평가가 왜곡될 수 있습니다. K-Fold 교차 검증은 데이터를 K개의 부분(fold)으로 나누고, 각 폴드가 한 번씩 테스트 세트가 되도록 K번의 훈련과 평가를 반복합니다. 이 점수들의 평균을 최종 성능으로 사용하여 더 신뢰할 수 있는 평가를 제공합니다.

In [ ]:
# 기본 SVC 모델 생성 (튜닝 전)
svc_base = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)

# 5-Fold 교차 검증 수행
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(svc_base, X_train, y_train, cv=kfold, scoring='accuracy')

print(f"Cross-validation scores: {scores}")
print(f"Average CV score: {scores.mean():.4f}")
print(f"CV score standard deviation: {scores.std():.4f}")

### (3) 그리드 서치를 이용한 하이퍼파라미터 최적화
`GridSearchCV`는 우리가 지정한 하이퍼파라미터 값들의 모든 가능한 조합에 대해 교차 검증을 수행하여, 가장 높은 평균 성능을 보인 조합을 찾아줍니다. 이 과정은 최적의 모델을 찾는 가장 확실한 방법 중 하나입니다.

In [ ]:
# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'C': [0.1, 1, 10, 50, 100],        # 규제 강도
    'gamma': [0.001, 0.01, 0.1, 1], # 커널의 영향 범위
    'kernel': ['rbf']
}

# GridSearchCV 객체 생성
# cv=5: 5-Fold 교차 검증으로 각 조합을 평가
# n_jobs=-1: 사용 가능한 모든 CPU 코어를 사용하여 병렬 처리
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, verbose=2, n_jobs=-1, scoring='accuracy')

# 그리드 서치 실행 (훈련 데이터에 대해)
grid_search.fit(X_train, y_train)

print("\n--- Grid Search Complete ---")
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy score: {grid_search.best_score_:.4f}")

### (4) 최적 모델로 최종 평가
그리드 서치를 통해 찾은 최적의 하이퍼파라미터로 모델을 최종 확정하고, 분리해 두었던 테스트 세트로 마지막 성능 평가를 수행합니다.

In [ ]:
# 그리드 서치가 찾은 최적의 모델
best_model = grid_search.best_estimator_

# 테스트 세트로 최종 예측 및 평가
y_pred = best_model.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)

print(f"\nAccuracy on the hold-out test set: {final_accuracy:.4f}")